In [1]:
import numpy as np
from main_functions_convnets import *

In [2]:
def padding(X, pad=0):
    """ A function which pads a tensor of dimension 4, being (m, height, width, number_channels) the form of the 
    tensor. I want to pad along the axis of height and width. """
    
    X_pad = np.pad(X,((0,0), (pad,pad), (pad,pad), (0,0)), 'constant', constant_values=0)
    
    return X_pad

pad= 1
X = np.ones([3,2,2,3])
X_pad = padding(X)

In [3]:
def convolution(A_prev_sample, kernel, stride=1, padding='valid'):
    """A function that implements a convolution of a sample of the dataset for one filter."""
    
    #First we calculate what is the height and width of the input. We consider that the input is given by the shape
    # (height, width, depth)
    height_input, width_input, depth = A_prev_sample.shape
    
    #Now we calculate f from the kernel. The kernel (o filter) is given with dimensions (f,f, number of channels)
    f, _, kernel_depth = kernel.shape
    
    # The padding is valid if we don't fill the image with 0's on its borders to manipulate dimensions.

    if padding == 'valid':
        pad = 0
    # If the padding is same it means I am calculating a pad so the input is the same as the output.    
    elif padding == 'same':
        pad = int((f-1)/2)
    
    # Here we calculate the shape of the output and we creatr
    
    height_output, width_output = int((height_input + 2*pad - f)/stride + 1), int((width_input + 2*pad - f)/stride + 1)
    # I create the output of the convolution after having considered f, the stride and the padding.
    conv_output = np.zeros([height_output, width_output])
    
    
    # Now we go in a for loop going through columns and the going down
    
    for height in range(height_output):
        # Now we go through the columns
        for width in range(width_output):
            
            #The current portion of the convolution will be image[height:(height+f), width:(width+f), :]
            current_portion = A_prev_sample[(height*stride):(height*stride+f), (width*stride):(width*stride+f),:]
            
            #Now I do the convolution operation throughout the whole input calling the function from the
            # main_functions_conv script.
            conv_output[height, width] = basic_convolution(current_portion, kernel)
                                         
    return conv_output
    

In [4]:
# Let's make a function that makes the convolution operation in a layer on all the filters and all the samples.
def convolution_forward(A_prev, W, stride=1, padding='valid'):
    """A function that takes the m samples of A_prev, all the filters from W and does an iteration in one
    layer of a convolution."""
    
    # First we calculate the dimensions of A_prev --> (m, n_H_prev, n_W_prev, n_c_prev)
    m, n_H_prev, n_W_prev, n_C_prev_A_prev = A_prev.shape
    
    # Now we calculate the dimensions of the filters W --> (f, f, n_c_prev, n_c)
    f, _, n_C_prev_W, n_C = W.shape
    
    #If the padding is valid, then the pad is equals to 0.
    if padding == 'valid':
        pad = 0
    
    # Now we assert that the number of channels of A n_c_prev and of filters are the same.
    assert (n_C_prev_A_prev == n_C_prev_W), "The number of channels of the previous layer must be the same\
    in the kernels as well as in the activation A_prev."
    
    # Now we calculate the height and width of the output.
    
    n_H, n_W = int((n_H_prev + 2*pad - f)/stride + 1),\
        int((n_W_prev + 2*pad - f)/stride + 1)
     
    # We now have the dimensions of the output. I will call it Z.
    Z = np.zeros([m, n_H, n_W, n_C])
    
    # Now we go through the m samples and the number of channels n_C in the layer:
    for i in range(m):
        # We go to the i sample of the dataset
        A_prev_sample = A_prev[i]
        
        for c in range(n_C):
            # Now let's get each of the filters of the network and apply the convolution. We have tp slice the
            # last dimension so we get the kernel of dimensions (f,f, n_C). This is W[:,:,:,c]
            kernel = W[:,:,:,c]
            # Once we get the kernel of channel c we can do the convolution to the sample and we can insert it
            # in the i'th sample of the network in the channel c.
            Z[i,:,:,c] = convolution(A_prev_sample, kernel, stride, padding)
            
    #Once convolution is made through the m samples and the n_C channels we have obtained the convolution in that 
    # layer and we can return Z.
    
    return Z
    

In [ ]:
np.random.seed(1)
f = 4
n_C_prev = 3
n_C = 2
m = 3
n_H_prev = 10
n_W_prev = 7

W = np.random.randint(low=0, high=3, size=(f, f, n_C_prev, n_C))
A_prev = np.random.randint(low=0, high=2, size=(m, n_H_prev, n_W_prev, n_C_prev))
print('A_prev is: ', A_prev, '\n')
print("Filters of the layer: ", W, '\n')
kernel = W[:,:,:,0]
print("Filter number 0: ", kernel, '\n')

# Let's check that the convolution operation does not give an error.
print('The convolution obtained is: ',convolution_forward(A_prev, W, stride=1, padding='valid'), '\n')

In [14]:
# Starting the operations of max pooling and average pooling on one channel.
def pooling_sample(A_prev_sample, f, pooling_type='max', stride=1):
    """A function  that makes the pooling operation for one sample. By default it makes the max pooling operation."""
    #As we have one sample, we have a 3D array with shape (n_H_prev, n_W_prev, n_C_prev). We can assert this
    # to make sure we give the right arguments.
    assert (len(A_prev_sample.shape) == 3), "The A_prev_sample must be a 3D array"
    
    n_H_prev, n_W_prev, n_C_prev = A_prev_sample.shape
    
    # Now we get the dimensions obtained with the f and stride:
    n_H = get_n_H(n_H_prev, f, stride)
    n_W = get_n_W(n_W_prev, f, stride)
    return n_H

    # Now we can create the output of the sample with dimensions height, width and the number of channels are the
    # same as in the input given.
    
    Z_sample = np.zeros(n_H, n_W, n_C_prev)
    
    # And now we can operate through the sample
    for c in range(n_C_prev):
        # We get the slice of Z in the channel c
        Z_sample_channel = Z_sample[:,:,c]
        for h in range(n_H):
            for w in range(n_W):
                # Now we need to iterate through the matrix in the channel c considering the stride and f:
                current_portion = Z_sample_channel[(h*stride):(h*stride+f),\
                                                   (w*stride):(w*stride+f),:]
                # Now we can extract the max or the mean of that portion depending if we want to do a max-pooling
                # or average pooling.
                if pooling_type.lower() == 'max':
                    Z_sample[h, w, c] = np.max(current_portion)
                elif pooling_type.lower() == 'average':
                    Z_sample[h, w, c] = np.mean(current_portion)
    
    # Once done in all the channels, we can return Z_sample
    return Z_sample
                
    

In [ ]:
np.random.seed(1)
n_H_prev = 3
n_W_prev = 4
n_C_prev = 4
c = 0
M = np.random.randint(low=0, high=3, size=(n_H_prev, n_W_prev, n_C_prev))
print('Sample: ', M, '\n')

M_channel =  M[:,:,c]
print('Channel ' + str(c) +': ', M[:,:,c])
M_channel.mean()

In [9]:
# Now we are going to load some data of cats and dogs to try our algorithm.
# I first import the libraries I need to interact with the directories.
import os
import matplotlib.pyplot as plt
import matplotlib.image as img
import cv2

current_dir = '/home/inapifer/Desktop/Deep_Learning'
i = 1000

rute_to_sample_cat = 'Data_cats_dogs/training_set/cats/cat.' + str(i) + '.jpg'


abs_rute_cat = os.path.join(current_dir, rute_to_sample_cat)

cat_matrix = img.imread(abs_rute_cat)
